In [54]:
import openai
import panel as pn
import os
import base64
import json
from openai import OpenAI
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from bs4 import BeautifulSoup

In [55]:
openai.__version__

'1.101.0'

In [56]:
with open ('/content/drive/MyDrive/APi_Keys/API_Key_Open_AI.txt') as f:
  api_key = f.readline().strip()

In [57]:
client = OpenAI(api_key=api_key)

In [65]:
Credential = '/content/drive/MyDrive/APi_Keys/client_secret_.apps.googleusercontent.com.json'
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

## Funciones

In [59]:
spinner = pn.indicators.LoadingSpinner(value=False, width=30, height=30)  # ⏳
status  = pn.pane.Markdown("", sizing_mode="stretch_width")               # línea de estado opcional

def with_busy(fn):
    def wrapper(event):
        spinner.value = True
        status.object = "⏳ Procesando..."
        try:
            return fn(event)
        finally:
            spinner.value = False
            status.object = ""
    return wrapper

In [60]:
def obtener_completion(mensajes, model = 'gpt-4o-mini'):
  respuesta = client.chat.completions.create(
      model = model,
      messages = mensajes,
      temperature = 0,
  )
  return respuesta.choices[0].message.content

In [61]:
def obtener_correos(remitente):
  creds = None
  if os.path.exists('/content/drive/MyDrive/APi_Keys/token.json'):
      creds = Credentials.from_authorized_user_file('/content/drive/MyDrive/APi_Keys/token.json', SCOPES)
  try:
      correos = """"""
      service = build('gmail', 'v1', credentials=creds)
      results = service.users().messages().list(userId='me', labelIds=['INBOX']).execute()
      for msg in results.get('messages',[]):
          message = service.users().messages().get(userId='me', id=msg['id']).execute()
          headers = message['payload'].get('headers',[])
          from_header = next((h['value'] for h in headers if h['name'] == 'From'), None)
          if from_header and remitente.strip().lower() in from_header.lower():
            parts = message['payload'].get('parts',[])
            for part in parts:
              if part['mimeType'] == 'text/plain':
                datos = part['body'].get('data')
                if datos:
                  decoded = base64.urlsafe_b64decode(datos).decode('utf-8')
                  correos += f"\n\n--- Correo ---\n{decoded}\n"

      if not correos:
          print('No se encontraron correos del remitente especificado')
      return correos
  except HttpError as error:
      print(f'Ocurrió un error de la API: {error}')


In [62]:
@with_busy
def end_chat(event):
  Interactive_conversation.append(pn.pane.Alert("Chat terminado por el usuario", alert_type = 'success'))
  farewell_ctx = [{
        'role':'system',
        'content': "Despídete del usuario de forma breve y amable. No hagas nada más."
    }]
  response = obtener_completion(farewell_ctx)
  context.append({'role':'assistant', 'content':f"{response}"})
  Interactive_conversation.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600,
                              styles={'background-color':'#F6F6F6','padding':'10px','border-radius':'5px'}))
    )

In [63]:
@with_busy
def collect_messages(event=None):
    prompt = inp.value
    inp.value = ''
    context.append({'role':'user', 'content': prompt})
    try:
        respuesta = obtener_completion(context)

        # Si la respuesta es JSON, la convierte y usa
        try:
            resultado = json.loads(respuesta)
            remitente = resultado.get("Remitente")
            accion = resultado.get("Accion_a_realizar")

            context.pop(0)  # Eliminamos el primer mensaje 'system'
            correos = obtener_correos(remitente)
            context.append({'role':'user','content':f"Realiza la siguiente accion:{accion} sobre los siguientes correos: {correos}"})
            respuesta = obtener_completion(context)
            resultado_final = f"Se obtuvo lo siguiente para {remitente}:\n\n{accion}\n\n--- Accion ---\n\n{respuesta}"
            Interactive_conversation.append(
                pn.Row('Assistant:', pn.pane.Markdown(resultado_final, width=600,
                                      styles={'background-color':'white','padding':'10px','border-radius':'5px'}))
            )
        except Exception as e:
            Interactive_conversation.append(
                pn.Row('Assistant:', pn.pane.Markdown(respuesta, width=600,
                                      styles={'background-color':'white','padding':'10px','border-radius':'5px'}))
            )
    except Exception as e:
        Interactive_conversation.append(pn.Row('Error:', pn.pane.Markdown(str(e), width=600)))


In [64]:
pn.extension(notifications = True)
panels = []
context = [ {'role':'system', 'content':
"""
Debes solictar al usuario la direccion de correo que quiere analizar y la tarea que quiere que se haga con los correos de esta direccion en el mismo prmopt.
Con esta informacion, debes generar un JSON con las
siguientes claves, no debes generar nada mas.
"Remitente":<direccion de correo dada por el usuario>,
"Accion_a_realizar:<Accion a realizar con los correos electronicos del Remitente>"
"""
} ]

inp = pn.widgets.TextInput(placeholder = 'Escribe tu mensaje aquí...')
button_conversation = pn.widgets.Button(name = 'Enviar')
button_end_chat = pn.widgets.Button(name = 'Terminar Chat')
Interactive_conversation = pn.Column(sizing_mode="stretch_width")

button_conversation.on_click(collect_messages)
button_end_chat.on_click(end_chat)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation, button_end_chat, spinner),
    Interactive_conversation,  # <- montas el contenedor directo
    sizing_mode="stretch_both"
)

dashboard


/tmp/ipython-input-800968146.py:1: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension(notifications = True)


NotificationArea()

Column(sizing_mode='stretch_both')
    [0] TextInput(placeholder='Escribe tu mensaje a...)
    [1] Row
        [0] Button(name='Enviar')
        [1] Button(name='Terminar Chat')
        [2] LoadingSpinner(height=30, width=30)
    [2] Column(sizing_mode='stretch_width')